In [63]:
from scipy.io import loadmat, savemat
import os
import numpy as np
from tqdm import tqdm
import pandas as pd

In [64]:
rows=['p01','p02','p03','p04','p05','p06','p07','p08','p09','p10','p11','p12','p13','p14','p15','p16','p17','p18','p19','p20','p21','p22','p23','p24','p25','p26','p27','p28','p29','p30','p31','p32','p33','p34','p35','p36','p37','p38','p39','p40']

In [65]:
path = './flow/Stat_20240531_BL_180_10_01'
mat_files = [f for f in os.listdir(path) if f.endswith('.mat')]
means = np.zeros((40,21))
rio = np.zeros((3,21))
i,l=0,0

for f in tqdm((mat_files)):   
    if (0.0<= float(f[-10:-4]) <= 180) and f.startswith('Stat_20240531_BL_180_10_01_cDAQ'):
        filepath = path + '/' +f
        f1 = loadmat(filepath)
        for j,(keys,values) in enumerate(f1.items()):
            if keys.startswith('cDAQ') and j<40:
                means[j,i]=np.mean(values)
        i+=1
    
    if (0.0<= float(f[-10:-4]) <= 180) and f.startswith('Stat_20240531_BL_180_10_01_RIO'):
        filepath = path + '/' +f
        f1 = loadmat(filepath)
        k=0
        for keys,values in f1.items():
            if keys.startswith('AI00'):
                rio[k,l]=np.mean(values)
                k+=1
            if keys.startswith('Dens') or keys.startswith('Sam'):
                rio[k,l]=float(values)
                k+=1
                if k==3:
                    break
        l+=1   

100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [00:01<00:00, 44.55it/s]


In [66]:
no_flow = np.empty((40,1))
f1 = loadmat('./noflow/Stat_20240531_BL_180_10_01/Stat_20240529_BL_180_20_01_cDAQ_Group00000_Group00000_No_Movement.mat')
for j,(keys,values) in enumerate(f1.items()):
    if keys.startswith('cDAQ') and j<40:
        no_flow[j,0]=np.mean(values)

In [67]:
means1=means-no_flow

means_df = pd.DataFrame(means1,index=rows,columns=['AoA_' + str(i) for i in np.arange(-10, 11, 1)])

means_df.to_csv('means_df.csv')

In [68]:
# Kalibrierung!!!
path1 = './Kalibration/20240326/20240326_CalNeg/'
path2 = './Kalibration/20240326/20240326_CalPos//'
cals = np.zeros((42,12))

def cal(index,path,mat):
    mat_files = [f for f in os.listdir(path) if f.endswith('.mat')]

    for f in tqdm((mat_files)):   
        filepath = path + '/' +f
        f1 = loadmat(filepath)
        for j,(keys,values) in enumerate(f1.items()):
            if keys=='x':

                break
            mat[j,index] = np.mean(values)
        index += 1
    
    return index,mat
i,cals=cal(0,path1,cals)
i,cals=cal(i,path2,cals)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 54.55it/s]


In [69]:
ind = ['p' + str(i) for i in np.arange(1,41,1)] + ['bara','cal84']
means_df = pd.DataFrame(cals,index=ind,columns=['X = ' + str(i) for i in list(np.arange(0,-1001,-200))+list(np.arange(0,1001,200))])
means_df.to_csv('cals_df.csv')